In [1]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'emogen2/tensorflow2/one/1:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-models-data%2F53721%2F64424%2Fbundle%2Farchive.tar.gz%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240613%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240613T132419Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3Da274f4e22fc82aa42ef2bd6731c6d2033b1d6ecf9240ba2a39989384ef7f6a49300e2c38c10c4cb235078f88bd815acf60b8249e16ce719a95388786cfc98f5d97802a3cd7824d16ecdf6acec387a6d0b223959fda03c3a0e95e1f0dfea8fefc83e4db5de89a7dd1fb3b241c94b2c05f53d6546f4292342b19f9afdb98a4f524365e3a2a7a9b7dbec8c1030d23b5a436a406a9fef7029251fd1521670b2f364fd2634c412170fd85cb622978eed2ff3348ed258ddabfd95fd72984968aa01802b4557768f3c6868c7b72f5c53fa5a152e8a9fea3f55285f1d0715b254b38fb0a338b0a135635f4690d81724ce333ac67c203165224b401e17f9296c433be87bc,yolov8/pytorch/open-image-v8/1:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-models-data%2F50333%2F60178%2Fbundle%2Farchive.tar.gz%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240613%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240613T132420Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D1e6139bd454ddc39527c6e4062138f8b3b2ce1f8a454ae59f4952c82199e4cd67c811b5d4214568f25602af5657b683f1e7624683aa23c6324e4b48f3af49be303a8a6e11a4e64318078671019067e9154e3e1be9b615d3d3a0030953e66facbb9392332a34cc69bea67d7d6234da21d6ef240e8a779e787c3c1c6858593e238a2f24e667ded02600c6feffb78195034cc8bc2457a925790469787bf3d2299179624020d0344aad29315c5f9bab2cb9c2f860fb22c836c6ea9093f32f979eb7dd5420485c25b919581e693f496031c7b17c9acf3a08de1d02dcc353ef91c34510a6f872894767485aa0b0da00565f63aa90e2dd32eb5f3ede6a401c8ffb1b2a7'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


[==================================================] 203010682 bytes downloaded
Downloaded and uncompressed: emogen2/tensorflow2/one/1
[==================================================] 21264265 bytes downloaded
Downloaded and uncompressed: yolov8/pytorch/open-image-v8/1
Data source import complete.


In [2]:
!pip install ultralytics lapx easyocr pytube dlib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 780.6/780.6 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 908.3/908.3 kB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 12.5 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 

In [3]:
import logging
import sys
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Activation, add, Dense, Flatten, Dropout, Conv2D, AveragePooling2D, BatchNormalization
from tensorflow.keras.regularizers import l2
from tensorflow.keras import backend as K
sys.setrecursionlimit(2 ** 20)
np.random.seed(2 ** 10)


class WideResNet:
    def __init__(self, image_size, depth=16, k=8):
        self._depth = depth
        self._k = k
        self._dropout_probability = 0
        self._weight_decay = 0.0005
        self._use_bias = False
        self._weight_init = "he_normal"

        if K.image_data_format() == "channels_first":
            logging.debug("image_dim_ordering = 'th'")
            self._channel_axis = 1
            self._input_shape = (3, image_size, image_size)
        else:
            logging.debug("image_dim_ordering = 'tf'")
            self._channel_axis = -1
            self._input_shape = (image_size, image_size, 3)

    # Wide residual network http://arxiv.org/abs/1605.07146
    def _wide_basic(self, n_input_plane, n_output_plane, stride):
        def f(net):
            # format of conv_params:
            #               [ [kernel_size=("kernel width", "kernel height"),
            #               strides="(stride_vertical,stride_horizontal)",
            #               padding="same" or "valid"] ]
            # B(3,3): orignal <<basic>> block
            conv_params = [[3, 3, stride, "same"],
                           [3, 3, (1, 1), "same"]]

            n_bottleneck_plane = n_output_plane

            # Residual block
            for i, v in enumerate(conv_params):
                if i == 0:
                    if n_input_plane != n_output_plane:
                        net = BatchNormalization(axis=self._channel_axis)(net)
                        net = Activation("relu")(net)
                        convs = net
                    else:
                        convs = BatchNormalization(axis=self._channel_axis)(net)
                        convs = Activation("relu")(convs)

                    convs = Conv2D(n_bottleneck_plane, kernel_size=(v[0], v[1]),
                                          strides=v[2],
                                          padding=v[3],
                                          kernel_initializer=self._weight_init,
                                          kernel_regularizer=l2(self._weight_decay),
                                          use_bias=self._use_bias)(convs)
                else:
                    convs = BatchNormalization(axis=self._channel_axis)(convs)
                    convs = Activation("relu")(convs)
                    if self._dropout_probability > 0:
                        convs = Dropout(self._dropout_probability)(convs)
                    convs = Conv2D(n_bottleneck_plane, kernel_size=(v[0], v[1]),
                                          strides=v[2],
                                          padding=v[3],
                                          kernel_initializer=self._weight_init,
                                          kernel_regularizer=l2(self._weight_decay),
                                          use_bias=self._use_bias)(convs)

            # Shortcut Connection: identity function or 1x1 convolutional
            #  (depends on difference between input & output shape - this
            #   corresponds to whether we are using the first block in each
            #   group; see _layer() ).
            if n_input_plane != n_output_plane:
                shortcut = Conv2D(n_output_plane, kernel_size=(1, 1),
                                         strides=stride,
                                         padding="same",
                                         kernel_initializer=self._weight_init,
                                         kernel_regularizer=l2(self._weight_decay),
                                         use_bias=self._use_bias)(net)
            else:
                shortcut = net

            return add([convs, shortcut])

        return f


    # "Stacking Residual Units on the same stage"
    def _layer(self, block, n_input_plane, n_output_plane, count, stride):
        def f(net):
            net = block(n_input_plane, n_output_plane, stride)(net)
            for i in range(2, int(count + 1)):
                net = block(n_output_plane, n_output_plane, stride=(1, 1))(net)
            return net

        return f

#    def create_model(self):
    def __call__(self):
        logging.debug("Creating model...")

        assert ((self._depth - 4) % 6 == 0)
        n = (self._depth - 4) / 6

        inputs = Input(shape=self._input_shape)

        n_stages = [16, 16 * self._k, 32 * self._k, 64 * self._k]

        conv1 = Conv2D(filters=n_stages[0], kernel_size=(3, 3),
                              strides=(1, 1),
                              padding="same",
                              kernel_initializer=self._weight_init,
                              kernel_regularizer=l2(self._weight_decay),
                              use_bias=self._use_bias)(inputs)  # "One conv at the beginning (spatial size: 32x32)"

        # Add wide residual blocks
        block_fn = self._wide_basic
        conv2 = self._layer(block_fn, n_input_plane=n_stages[0], n_output_plane=n_stages[1], count=n, stride=(1, 1))(conv1)
        conv3 = self._layer(block_fn, n_input_plane=n_stages[1], n_output_plane=n_stages[2], count=n, stride=(2, 2))(conv2)
        conv4 = self._layer(block_fn, n_input_plane=n_stages[2], n_output_plane=n_stages[3], count=n, stride=(2, 2))(conv3)
        batch_norm = BatchNormalization(axis=self._channel_axis)(conv4)
        relu = Activation("relu")(batch_norm)

        # Classifier block
        pool = AveragePooling2D(pool_size=(8, 8), strides=(1, 1), padding="same")(relu)
        flatten = Flatten()(pool)
        predictions_g = Dense(units=2, kernel_initializer=self._weight_init, use_bias=self._use_bias,
                              kernel_regularizer=l2(self._weight_decay), activation="softmax",
                              name="pred_gender")(flatten)
        predictions_a = Dense(units=101, kernel_initializer=self._weight_init, use_bias=self._use_bias,
                              kernel_regularizer=l2(self._weight_decay), activation="softmax",
                              name="pred_age")(flatten)
        model = Model(inputs=inputs, outputs=[predictions_g, predictions_a])

        return model

In [4]:
from pytube import YouTube

video_url = 'https://www.youtube.com/watch?v=o4Zd-IeMlSY'

yt = YouTube(video_url)

video_stream = yt.streams.get_highest_resolution()

output_path = 'downloaded_video.mp4'

video_stream.download(filename=output_path)

print(f"Video downloaded and saved as {output_path}")

Video downloaded and saved as downloaded_video.mp4


In [7]:
from tensorflow.keras.models import load_model
from ultralytics import YOLO
import easyocr
import torch
import time
import dlib
import cv2
from tensorflow.keras.preprocessing.image import img_to_array
import sys
import io
from transformers import pipeline

class WrapperClass:
    def __init__(self, img_size, depth, k, margin):
        self.img_size = img_size
        self.margin = margin
        self.ag_model = WideResNet(img_size, depth=depth, k=k)()
        self.ag_model.load_weights("/kaggle/input/emogen2/tensorflow2/one/1/weights.28-3.73.hdf5")
        self.emotion_model = load_model("/kaggle/input/emogen2/tensorflow2/one/1/emotion_little_vgg_2.h5")
        self.yolo = YOLO("/kaggle/input/yolov8/pytorch/open-image-v8/1/yolov8s-oiv7.pt")
        self.easyocr = easyocr.Reader(['en'], gpu=torch.cuda.is_available())
        self.emotion_classes = {0:"Angry", 1:"Disgust", 2:"Fear", 3:"Happy", 4:"Sad", 5:"Surprise", 6:"Neutral"}
        self.detector = dlib.get_frontal_face_detector()
        self.corrector = pipeline('text2text-generation', model='vennify/t5-base-grammar-correction')

    def detection(self, frame):
        return self.detector(frame, 1)

    def preprocess_face(self, img_h, img_w, detected, faces, frame):
        preprocessed_faces_emo = []
        if len(detected) > 0:
            for i, d in enumerate(detected):
                x1, y1, x2, y2, w, h = d.left(), d.top(), d.right() + 1, d.bottom() + 1, d.width(), d.height()
                xw1 = max(int(x1 - self.margin * w), 0)
                yw1 = max(int(y1 - self.margin * h), 0)
                xw2 = min(int(x2 + self.margin * w), img_w - 1)
                yw2 = min(int(y2 + self.margin * h), img_h - 1)
                faces[i, :, :, :] = cv2.resize(frame[yw1:yw2 + 1, xw1:xw2 + 1, :], (self.img_size, self.img_size))
                face =  frame[yw1:yw2 + 1, xw1:xw2 + 1, :]
                face_gray_emo = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
                face_gray_emo = cv2.resize(face_gray_emo, (48, 48), interpolation = cv2.INTER_AREA)
                face_gray_emo = face_gray_emo.astype("float") / 255.0
                face_gray_emo = img_to_array(face_gray_emo)
                face_gray_emo = np.expand_dims(face_gray_emo, axis=0)
                preprocessed_faces_emo.append(face_gray_emo)

        return preprocessed_faces_emo, faces

    def predict_ages(self, detected, faces):
        if len(detected) > 0:
            results = self.ag_model.predict(np.array(faces))
            predicted_genders = results[0]
            predicted_ages = np.argsort(results[1])[0][-5:]
            age_range = (min(predicted_ages), max(predicted_ages))

            return predicted_genders, age_range

        return None, None

    def predict_emo(self, detected, preprocessed_faces_emo):
        emo_labels = []
        for i, d in enumerate(detected):
            preds = self.emotion_model.predict(preprocessed_faces_emo[i])[0]
            e = self.emotion_classes[preds.argmax()]
            emo_labels.append(e)
        return emo_labels

    def yolo_pred(self, frame):
        res = self.yolo(frame)
        return [(i['name'], i['box']) for j in res for i in j.summary()]

    def correct_text(self, text):
        return self.corrector(text)[0]['generated_text']


    def vision_pred(self, frame):
        preprocessed_faces_emo = []
        input_img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        img_h, img_w, _ = np.shape(input_img)
        detected = self.detector(frame, 1)
        faces = np.empty((len(detected), self.img_size, self.img_size, 3))
        preprocessed_faces_emo, faces = self.preprocess_face(img_h, img_w, detected, faces, frame)
        gender, age = self.predict_ages(detected, faces)
        emo_labels = self.predict_emo(detected, preprocessed_faces_emo)
        print(gender, age, emo_labels)
        result = self.yolo_pred(frame)
        print(result)

    def ocr_pred(self, frame):
        result = self.easyocr.readtext(frame)
        detected_text = ""
        for item in result:
            detected_text += " " + item[1]
        corrected_text = self.correct_text(detected_text)
        print(corrected_text)

    def make_prediction(self, video_path, func):
        a = time.time()
        output_video_path = 'output.mp4'
        cap = cv2.VideoCapture(video_path)
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        fps = int(cap.get(cv2.CAP_PROP_FPS))
        frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        frame_count = 0

        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break

            if frame_count % 30 != 0:
                frame_count += 1
                continue
            if frame_count == 1500:
              break

            print(frame_count)
            func(frame)
            frame_count += 1

        cap.release()
        cv2.destroyAllWindows()
        b = time.time()
        print(b - a)

    def __call__(self, video_path, func):
        if func == "see":
            self.make_prediction(video_path, self.vision_pred)
        elif func == "ocr":
            self.make_prediction(video_path, self.ocr_pred)

In [8]:
a = WrapperClass(64, 16, 8, 0.3)
a("/content/downloaded_video.mp4", "see")

0
1/1 [==============================] - 0s 390ms/step
[[  0.0013018      0.9987]] (26, 32) ['Happy']

0: 384x640 1 Chair, 1 Clothing, 1 Houseplant, 1 Human face, 1 Man, 1 Microphone, 620.9ms
Speed: 6.1ms preprocess, 620.9ms inference, 29.7ms postprocess per image at shape (1, 3, 384, 640)
[('Human face', {'x1': 561.53149, 'y1': 171.50504, 'x2': 758.02905, 'y2': 477.70853}), ('Man', {'x1': 228.54907, 'y1': 96.98566, 'x2': 998.32617, 'y2': 717.63141}), ('Chair', {'x1': 798.90979, 'y1': 432.22934, 'x2': 1139.91052, 'y2': 719.86267}), ('Houseplant', {'x1': 1095.89551, 'y1': 325.46219, 'x2': 1280.0, 'y2': 719.4231}), ('Microphone', {'x1': 234.26419, 'y1': 421.36841, 'x2': 495.71494, 'y2': 639.00708}), ('Clothing', {'x1': 241.07996, 'y1': 407.42761, 'x2': 993.14587, 'y2': 717.78442})]
30
1/1 [==============================] - 0s 28ms/step
[[  0.0067195     0.99328]] (22, 32) ['Happy']

0: 384x640 1 Chair, 1 Clothing, 1 Houseplant, 1 Human face, 1 Man, 1 Microphone, 412.9ms
Speed: 5.1ms prep

In [10]:
a("/content/downloaded_video.mp4", "ocr")

0


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1168: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(



30

60

90
Car 0.941 cc7_0 car 0.69; Car 0.90 UG
120
Cat 0 89 car 0.81ar Ocar 0 icar_0.94
150
71aaeae Ultrollytics YOLOv8 Ton
180
NEW TOHORROw is a new tohorrow.
210
Ultralytics ShAPI.
240
The new technology is ultralytical.
270
Is it possible?
300
Is it possible?
330
New to the forecast, the forecast is for the future.
360
IoRrow.
390
SHAPIMG L Inaney is a SHAPIMG L Inaney
420
New TQVOrROw is announcing a new TQVOrRO
450
YOLOv8 is a language designed to be fast, accurate, and easy
480
To request an Enterprise License, please browse the YOLOv8 Docs for details
510
Use CLI YOLOv8 may be used directly in the Command Line Interface (
540
For making predictions using trained YOLOv8 model on custom datasets, the following
570
IcnoRROw is a shaminc mew Ic
600
Fasie Shaping New Tomorrow!
630
Shading New Chorrow, the new town of Chorrow.
660
YolovR Inletenc "FPS: 67 person 0.90 JF'
690
86 pottod elont 044 chair 0.69_ c0l
720
92 potted_plont 0.69 chair 0.75_ 'TOHOR
750
ILJClorg Infrteno FPEILZ